In [36]:
import numpy as np
import numpy.typing as npt
import typing
import gymnasium as gym
from gymnasium import spaces
from typing import Callable, List, Dict, Tuple, Any

In [37]:
def BlackScholesPathSingle(
    S_0: float, 
    r: float, 
    sigma: float, 
    T: float, 
    N: int, 
    rng: np.random.Generator
):
    delta = T / N
    Z = rng.normal(loc=0, scale=1, size=(N))
    path = np.empty((N + 1), dtype=np.float64)
    path[0] = S_0
    for i in range(N):
        path[i + 1] = path[i] * np.exp((r - 0.5 * np.power(sigma, 2.0)) * delta + sigma * np.sqrt(delta) * Z[i])
    return path

class MultiAgentSimulationEnv(gym.Env):
    def _get_true_dividend(self, tick: int) -> float:
        return self.annualized_dividend_path[tick]
    
    def _get_agent_dividend_estimate(self, agent_id: int) -> float:
        tick = self.simulation.get_tick()
        annualized_dividend = self.annualized_dividend_path[tick]
        analyst_mean_error = self.agent_analyst_mean_error[agent_id]
        analyst_error = self.rng.normal(analyst_mean_error, 0.001)
        return min(annualized_dividend + analyst_error, 0.0)
    
    def __init__(self, agent_count: int):
        super().__init__()
        import python_modules.Server as Server
        self.Server = Server
        
        self.rng = np.random.default_rng()
        self.step_count = 1000
        
        self.cash_borrowing_cost = 0.04
        self.bond_interest_rate = 0.05
        self.bond_face_value = 100.0
        
        self.dividend_initial_value = 0.06
        self.dividend_growth_rate = 0.1
        self.dividend_volatility = 0.1
        self.annualized_dividend_path = BlackScholesPathSingle(
            self.dividend_initial_value, 
            self.dividend_growth_rate,
            self.dividend_volatility,
            1.0,
            self.step_count + 1,
            self.rng
        )
        
        self.MAX_STOCK_POSITION = 20
        self.MAX_BOND_POSITION = 30
        self.lambda_risk = 0.001
        
        self.initial_stock_price = self.step_count * self.dividend_initial_value
        
        self.cash = self.Server.GenericSecurities.MarginCash("CASH", self.cash_borrowing_cost, 0.0)
        self.bond = self.Server.GenericSecurities.GenericBond("BOND", "CASH", self.bond_interest_rate, self.bond_face_value) 
        self.stock = self.Server.GenericSecurities.DividendStock("STOCK", "CASH", dividend_function=lambda tick: self._get_true_dividend(tick))
        
        self.simulation = self.Server.GenericSimulation({
            "CASH": self.cash,
            "BOND": self.bond,
            "STOCK": self.stock    
        }, 1.0, self.step_count)
        
        self.cash_id = self.simulation.get_security_id("CASH")
        self.bond_id = self.simulation.get_security_id("BOND")
        self.stock_id = self.simulation.get_security_id("STOCK")
        
        self.agent_ids: set[int] = set()
        self.previous_portfolio_value: dict[int, float] = {}
        self.agent_analyst_mean_error: dict[int, float] = {}
        for agent_id in range(agent_count):
            id = self.simulation.add_user(f"AGENT_{agent_id}")
            self.agent_ids.add(id)
            self.previous_portfolio_value[id] = 0.0
            self.agent_analyst_mean_error[id] = self.rng.uniform(-0.01, 0.01)
            pass
        
        self.past_bond_midpoints: List[float] = [self.bond_face_value]
        self.past_stock_midpoints: List[float] = [self.initial_stock_price]
        self.observation_book_length = 5 # Top 5
        
        self.observation_spaces = {
            agent_id: spaces.Dict({
                "order_book_bond_asks": spaces.Box(low=0.0, high=np.inf, shape=(self.observation_book_length,2), dtype=np.float32), # (price, volume)
                "order_book_bond_bids": spaces.Box(low=0.0, high=np.inf, shape=(self.observation_book_length,2), dtype=np.float32), # (price, volume)
                "order_book_stock_asks": spaces.Box(low=0.0, high=np.inf, shape=(self.observation_book_length,2), dtype=np.float32), # (price, volume)
                "order_book_stock_bids": spaces.Box(low=0.0, high=np.inf, shape=(self.observation_book_length,2), dtype=np.float32), # (price, volume)
                "cash_borrowing_cost": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32),
                "bond_interest_rate": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32),
                "bond_face_value": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32),
                "bond_midpoint_price": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32),
                "stock_midpoint_price": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32),
                "bond_volatility_5": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32),
                "bond_volatility_20": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32),
                "bond_volatility_100": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32),
                "stock_volatility_5": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32),
                "stock_volatility_20": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32),
                "stock_volatility_100": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32),
                "bond_price_delta": spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32),
                "stock_price_delta": spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32),
                "current_expected_dividend": spaces.Box(low=0.0, high=np.inf, shape=(1,), dtype=np.float32), # The expected dividend per share we will receive if we own by the end of the current tick
                "remaining_bond_position": spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32), # The remaining position until the limit is reached (it can be breached using limit orders, that's why we also have negative)
                "remaining_stock_position": spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32),
                "open_orders_count": spaces.Box(low=0, high=np.inf, shape=(2,), dtype=np.int32), # (bond, stock)
                "positions": spaces.Box(low=-np.inf, high=np.inf, shape=(3,), dtype=np.float32), # (cash, bond, stock)
            }) for agent_id in self.agent_ids
        }

        self.RELATIVE_PRICE_BINS = 201
        self.action_spaces = {agent_id: gym.spaces.MultiDiscrete([
            4, # Order type: cancel (all own open limit orders), limit, market, (nothing),
            2, # Security: 0=bond, 1=stock
            2, # Side: 0=buy, 1=sell
            self.RELATIVE_PRICE_BINS, # Relative price (gets normalized to [-1$, 1$] inclusive)
            5  # Quantity: (1-5)
        ]) for agent_id in self.agent_ids}
        pass
    
    def _get_midpoints(self):
        bond_midpoint = self.bond_face_value
        if self.simulation.get_ask_count(self.bond_id) > 0 and self.simulation.get_bid_count(self.bond_id) > 0:
            bond_midpoint = (self.simulation.get_top_ask(self.bond_id).price + self.simulation.get_top_bid(self.bond_id).price) / 2
            
        stock_midpoint = self.initial_stock_price
        if self.simulation.get_ask_count(self.stock_id) > 0 and self.simulation.get_bid_count(self.stock_id) > 0:
            stock_midpoint = (self.simulation.get_top_ask(self.stock_id).price + self.simulation.get_top_bid(self.stock_id).price) / 2
        
        return (bond_midpoint, stock_midpoint)
    
    def _get_obs(self):
        bond_midpoint, stock_midpoint = self._get_midpoints()

        bond_bids, bond_asks = self.simulation.get_order_book(self.bond_id)
        stock_bids, stock_asks = self.simulation.get_order_book(self.stock_id)
        
        def book_to_array(orders: list):
            length = self.observation_book_length
            arr = np.zeros((length, 2), dtype=np.float32)
            for i in range(min(length, len(orders))):
                arr[i, 0] = orders[i].price
                arr[i, 1] = orders[i].volume
            return arr

        bond_ask_arr = book_to_array(bond_asks)
        bond_bid_arr = book_to_array(bond_bids)
        stock_ask_arr = book_to_array(stock_asks)
        stock_bid_arr = book_to_array(stock_bids)

        bond_midpoints_5 = self.past_bond_midpoints[-5:]
        bond_midpoints_20 = self.past_bond_midpoints[-20:]
        bond_midpoints_100 = self.past_bond_midpoints[-100:]
        stock_midpoints_5 = self.past_stock_midpoints[-5:]
        stock_midpoints_20 = self.past_stock_midpoints[-20:]
        stock_midpoints_100 = self.past_stock_midpoints[-100:]

        bond_volatility_5 = np.std(bond_midpoints_5) if len(bond_midpoints_5) >= 5 else 0.0
        bond_volatility_20 = np.std(bond_midpoints_20) if len(bond_midpoints_20) >= 20 else 0.0
        bond_volatility_100 = np.std(bond_midpoints_100) if len(bond_midpoints_100) >= 100 else 0.0
        
        stock_volatility_5 = np.std(stock_midpoints_5) if len(stock_midpoints_5) >= 5 else 0.0
        stock_volatility_20 = np.std(stock_midpoints_20) if len(stock_midpoints_20) >= 20 else 0.0
        stock_volatility_100 = np.std(stock_midpoints_100) if len(stock_midpoints_100) >= 100 else 0.0
        
        bond_price_delta = (self.past_bond_midpoints[-1] - self.past_bond_midpoints[-2]) if len(self.past_bond_midpoints) >= 2 else 0.0
        stock_price_delta = (self.past_stock_midpoints[-1] - self.past_stock_midpoints[-2]) if len(self.past_stock_midpoints) >= 2 else 0.0

        obs = {}

        for agent_id in self.agent_ids:
            portfolio = self.simulation.get_user_portfolio(agent_id)
            bond_position = portfolio[self.bond_id]
            stock_position = portfolio[self.stock_id]
            cash_position = portfolio[self.cash_id]

            open_bond_orders = len(self.simulation.get_all_open_user_orders(agent_id, self.bond_id))
            open_stock_orders = len(self.simulation.get_all_open_user_orders(agent_id, self.stock_id))

            remaining_bond_position = (self.MAX_BOND_POSITION - bond_position)
            remaining_stock_position = (self.MAX_STOCK_POSITION - stock_position)

            current_expected_dividend = self._get_agent_dividend_estimate(agent_id)

            obs[agent_id] = {
                "order_book_bond_asks": bond_ask_arr.copy(),
                "order_book_bond_bids": bond_bid_arr.copy(),
                "order_book_stock_asks": stock_ask_arr.copy(),
                "order_book_stock_bids": stock_bid_arr.copy(),
                "cash_borrowing_cost": np.array([self.cash_borrowing_cost], dtype=np.float32),
                "bond_interest_rate": np.array([self.bond_interest_rate], dtype=np.float32),
                "bond_face_value": np.array([self.bond_face_value], dtype=np.float32),
                "bond_midpoint_price": np.array([bond_midpoint], dtype=np.float32),
                "stock_midpoint_price": np.array([stock_midpoint], dtype=np.float32),
                "bond_volatility_5": np.array([bond_volatility_5], dtype=np.float32),
                "bond_volatility_20": np.array([bond_volatility_20], dtype=np.float32),
                "bond_volatility_100": np.array([bond_volatility_100], dtype=np.float32),
                "stock_volatility_5": np.array([stock_volatility_5], dtype=np.float32),
                "stock_volatility_20": np.array([stock_volatility_20], dtype=np.float32),
                "stock_volatility_100": np.array([stock_volatility_100], dtype=np.float32),
                "bond_price_delta": np.array([bond_price_delta], dtype=np.float32),
                "stock_price_delta": np.array([stock_price_delta], dtype=np.float32),
                "current_expected_dividend": np.array([current_expected_dividend], dtype=np.float32),
                "remaining_bond_position": np.array([remaining_bond_position], dtype=np.float32),
                "remaining_stock_position": np.array([remaining_stock_position], dtype=np.float32),
                "open_orders_count": np.array([open_bond_orders, open_stock_orders], dtype=np.int32),
                "positions": np.array([cash_position, bond_position, stock_position], dtype=np.float32),
            }
            pass

        return obs

    def reset(self, seed=None, options=None):
        self.simulation.reset_simulation()
        bond_midpoint, stock_midpoint = self._get_midpoints()
        self.past_bond_midpoints = [bond_midpoint]
        self.past_stock_midpoints = [stock_midpoint]
        for agent_id in self.agent_ids:
            self.previous_portfolio_value[agent_id] = 0.0
            self.agent_analyst_mean_error[agent_id] = self.rng.uniform(-0.01, 0.01)
            pass
        self.annualized_dividend_path = BlackScholesPathSingle(
            self.dividend_initial_value, 
            self.dividend_growth_rate,
            self.dividend_volatility,
            1.0,
            self.step_count + 1,
            self.rng
        )
        obs = self._get_obs()
        return obs, {}
    
    def step(self, actions: dict[int, npt.NDArray[np.int64]]):
        items = list(actions.items())
        self.rng.shuffle(items)
        
        bond_midpoint, stock_midpoint = self._get_midpoints()
        
        for agent_id, action in items:
            agent_portfolio = self.simulation.get_user_portfolio(agent_id)
            
            action_tuple = tuple(map(lambda x: x, action))
            
            match action_tuple:
                case (0, _, _, _, _): # Cancel all of the agent's limit orders
                    bond_orders = self.simulation.get_all_open_user_orders(agent_id, self.bond_id)
                    for bond_order in bond_orders:    
                        self.simulation.submit_cancel_order(agent_id, self.bond_id, bond_order)
                    stock_orders = self.simulation.get_all_open_user_orders(agent_id, self.stock_id)
                    for stock_order in stock_orders:
                        self.simulation.submit_cancel_order(agent_id, self.stock_id, stock_order)
                    pass
                case (1, security, side, relative_price_idx, quantity): # Limit order
                    quantity += 1 # Since the discrete starts at 0
                    # Technically the agent can go beyond the max positions using limit order:
                    # i.e. they submit a bunch but none is executed, then quickly all get executed
                    # resulting in them exceeding the position limit. In this case we only allow
                    # reduce to close their positions.
                    relative_price = -1.0 + (relative_price_idx) / (self.RELATIVE_PRICE_BINS - 1)  # From bin to cents
                    if security == 0:
                        # Bond
                        position = agent_portfolio[self.bond_id]
                        price = round(bond_midpoint + relative_price, 2)
                        if side == 0 and position + quantity <= self.MAX_BOND_POSITION:
                            # Buy
                            self.simulation.submit_limit_order(agent_id, self.bond_id, self.Server.OrderSide.BID, price, quantity)
                            pass
                        elif side == 1 and position - quantity >= -self.MAX_BOND_POSITION:
                            # Sell
                            self.simulation.submit_limit_order(agent_id, self.bond_id, self.Server.OrderSide.ASK, price, quantity)
                            pass
                        pass
                    elif security == 1:
                        # Stock
                        position = agent_portfolio[self.stock_id]
                        price = round(stock_midpoint + relative_price, 2)
                        if side == 0 and position + quantity <= self.MAX_STOCK_POSITION:
                            # Buy
                            self.simulation.submit_limit_order(agent_id, self.stock_id, self.Server.OrderSide.BID, price, quantity)
                            pass
                        elif side == 1 and position - quantity >= -self.MAX_STOCK_POSITION:
                            # Sell
                            self.simulation.submit_limit_order(agent_id, self.stock_id, self.Server.OrderSide.ASK, price, quantity)
                            pass
                        pass
                    pass
                case (2, security, side, _, quantity): # Market order
                    quantity += 1
                    # We allow agents to close positions even if outside limits
                    if security == 0:
                        # Bond
                        position = agent_portfolio[self.bond_id]
                        if side == 0 and position + quantity <= self.MAX_BOND_POSITION:
                            # Buy
                            self.simulation.submit_market_order(agent_id, self.bond_id, self.Server.OrderAction.BUY, quantity)
                            pass
                        elif side == 1 and position - quantity >= -self.MAX_BOND_POSITION:
                            # Sell
                            self.simulation.submit_market_order(agent_id, self.bond_id, self.Server.OrderAction.SELL, quantity)
                            pass
                        pass
                    elif security == 1:
                        # Stock
                        position = agent_portfolio[self.stock_id]
                        if side == 0 and position + quantity <= self.MAX_STOCK_POSITION:
                            # Buy
                            self.simulation.submit_market_order(agent_id, self.stock_id, self.Server.OrderAction.BUY, quantity)
                            pass
                        elif side == 1 and position - quantity >= -self.MAX_STOCK_POSITION:
                            # Sell
                            self.simulation.submit_market_order(agent_id, self.stock_id, self.Server.OrderAction.SELL, quantity)
                            pass
                        pass
                    pass
                case (3, _, _, _, _): # Do nothing
                    pass
                case unmatched:
                    print("Received unmatched case")
                    print(type(unmatched))
                    raise Exception("Received unmatched case.")
            pass
        
        result = self.simulation.do_simulation_step()
        bond_midpoint, stock_midpoint = self._get_midpoints()
        self.past_bond_midpoints.append(bond_midpoint)
        self.past_stock_midpoints.append(stock_midpoint)
        
        terminated = not result.has_next_step
        obs = self._get_obs()
        
        rewards = {}
        for agent_id in self.agent_ids:
            portfolio = self.simulation.get_user_portfolio(agent_id)
            cash = portfolio[self.cash_id]
            bond = portfolio[self.bond_id]
            stock = portfolio[self.stock_id]
            
            current_portfolio_value = cash + bond * bond_midpoint + stock * stock_midpoint
            previous_portfolio_value = self.previous_portfolio_value[agent_id]
            
            risk_penalty = self.lambda_risk * (bond ** 2 + stock ** 2)
            
            pnl_reward = current_portfolio_value - previous_portfolio_value
            rewards[agent_id] = np.clip(pnl_reward - risk_penalty, -10.0, 10.0)
            self.previous_portfolio_value[agent_id] = current_portfolio_value
            pass
        return obs, rewards, terminated, False, {}
    pass

In [38]:
from pettingzoo.utils import ParallelEnv

class TraderMarketParallelEnv(ParallelEnv):
    """
    PettingZoo ParallelEnv that simply *wraps* the existing MultiAgentSimulationEnv.

    * Agents are exposed as strings: "agent_0", "agent_1", …
    * Internally we keep the integer IDs used by MultiAgentSimulationEnv.
    * Observations/rewards stay Dict → nothing is flattened, privacy is kept.
    """

    metadata = {"name": "TraderMarketParallel_v0", "render_modes": []}

    # ---------- constructor -------------------------------------------------
    def __init__(self, agent_count: int = 20):
        super().__init__()
        self.render_mode = None
        
        # inner market simulation ------------------------------------------------
        self.inner = MultiAgentSimulationEnv(agent_count=agent_count)

        # build (int ↔ str) ID maps ---------------------------------------------
        self._int_ids           = sorted(list(self.inner.agent_ids))           # e.g. [7, 9, 12, …]
        self._name_from_int     = {i: f"agent_{k}" for k, i in enumerate(self._int_ids)}
        self._int_from_name     = {v: k for k, v in self._name_from_int.items()}

        # PettingZoo API fields --------------------------------------------------
        self.possible_agents    = list(self._name_from_int.values())           # static list
        self.agents             = self.possible_agents.copy()                 # active agents

        # Spaces (just re-use the inner ones) -----------------------------------
        self.observation_spaces: Dict[str, spaces.Space] = {
            self._name_from_int[i]: self.inner.observation_spaces[i] for i in self._int_ids
        }
        self.action_spaces: Dict[str, spaces.Space] = {
            self._name_from_int[i]: self.inner.action_spaces[i] for i in self._int_ids
        }

    # ---------- helpers -------------------------------------------------------
    def _translate_obs(self, obs_int: Dict[int, Any]) -> Dict[str, Any]:
        return {self._name_from_int[i]: ob for i, ob in obs_int.items()}

    def _translate_rewards(self, rew_int: Dict[int, float]) -> Dict[str, float]:
        return {self._name_from_int[i]: r for i, r in rew_int.items()}

    # ---------- PettingZoo parallel API ---------------------------------------
    def reset(self, *, seed: int | None = None, options: Dict | None = None):
        obs_int, info = self.inner.reset(seed=seed, options=options)
        self.agents = self.possible_agents.copy()
        obs   = self._translate_obs(obs_int)
        infos = {name: {} for name in self.agents}
        return obs, infos

    def step(self, actions: Dict[str, npt.NDArray]):
        # map string-keys → int-keys for the wrapped env
        int_actions = {self._int_from_name[n]: act for n, act in actions.items()}

        obs_i, rew_i, terminated, truncated, info_i = self.inner.step(int_actions)

        obs          = self._translate_obs(obs_i)
        rewards      = self._translate_rewards(rew_i)
        terminations = {name: terminated for name in self.agents}
        truncations  = {name: truncated  for name in self.agents}
        infos        = {self._name_from_int[i]: info_i.get(i, {}) for i in self._int_ids}

        # PettingZoo requires `agents` becomes [] once everyone is done
        if terminated or truncated:
            self.agents = []

        return obs, rewards, terminations, truncations, infos

    # ---------- optional extras ----------------------------------------------
    def render(self):                     # you can call self.inner.render() if you have one
        return None

    def close(self):
        if hasattr(self.inner, "close"):
            self.inner.close()

In [39]:
import supersuit as ss
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize

NUM_PARALLEL_ENVS = 8     # how many independent markets
AGENT_COUNT       = 20    # traders per market

# --------------------------------------------------------------------------
# 1. Raw PettingZoo parallel env
pz_env = TraderMarketParallelEnv(agent_count=AGENT_COUNT)

# 2. Convert → single SB-3 VecEnv (vector length = AGENT_COUNT)
sb3_vec_env = ss.pettingzoo_env_to_vec_env_v1(
    pz_env,
)

# ----------  !!! 2-line patch for SuperSuit 3.10.0 !!! --------------------
# SuperSuit 3.10.0's concat helper wants these two attributes:
sb3_vec_env.single_observation_space = sb3_vec_env.observation_space
sb3_vec_env.single_action_space      = sb3_vec_env.action_space
# --------------------------------------------------------------------------

# 3. Replicate that VecEnv to run NUM_PARALLEL_ENVS in parallel
vec_env = ss.concat_vec_envs_v1(
    sb3_vec_env,
    NUM_PARALLEL_ENVS,              # positional, not keyword!
    num_cpus=NUM_PARALLEL_ENVS,     # 0 ⇒ run in-process
    base_class="stable_baselines3",
)

# 4. Optional but recommended: normalise obs & rewards
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True)

# 5. Train a shared PPO policy (Dict observations → MultiInput network)
model = PPO(
    policy="MultiInputPolicy",
    env=vec_env,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    verbose=1,
)
model.learn(total_timesteps=500_000)

model.save("ppo_trader_shared_policy")
vec_env.save("vecnormalize.pkl")


TypeError: MakeCPUAsyncConstructor.<locals>.constructor() missing 2 required positional arguments: 'obs_space' and 'act_space'